In [ ]:
!nvidia-smi

Thu Dec  7 17:51:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


import libraries

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 400, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 400 (delta 95), reused 53 (delta 51), pack-reused 269
Receiving objects: 100% (400/400), 109.81 KiB | 13.73 MiB/s, done.
Resolving deltas: 100% (197/197), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.5 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.4/506.4 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  error: subprocess-exited-with-error

  × python setup.py egg_info did not run

In [ ]:
import cuml

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
from xgboost import XGBClassifier

In [ ]:
from cuml import LogisticRegression
from cuml import SVC
from cuml import KNeighborsClassifier
from cuml.ensemble import RandomForestClassifier
from cuml import LinearSVC

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score

# Civil

Loading dataset

In [ ]:
# loading dataset
dataset = pd.read_csv("/content/drive/MyDrive/LJP_2023/LJP_Dataset_Files/civil_documents/case_files_civil.csv")
print(len(dataset))
dataset.rename(columns = {'judgement':'text'}, inplace = True)
dataset = dataset[dataset['label'] != "Other" ]
dataset["label"] = dataset["label"].replace("Accepted", 1)
dataset["label"] = dataset["label"].replace("Rejected", 0)

#filter dataset
dataset.label.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset = dataset.dropna(axis=0, subset=['text'])
dataset = dataset.dropna(axis=0, subset=['label'])
dataset['label'] = dataset['label'].astype(int)
dataset.reset_index(drop=True, inplace=True)
dataset = dataset['label']
print(dataset) #should have only the label column

34543
0        1
1        1
2        1
3        1
4        0
        ..
28208    0
28209    1
28210    0
28211    0
28212    1
Name: label, Length: 28213, dtype: int64


<ipython-input-4-1d997c02033e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['label'] = dataset['label'].astype(int)


In [ ]:
y=dataset

Loading embeddings

In [ ]:
X=np.load("/content/drive/MyDrive/LJP_2023/Embeddings/Doc2Vec/Experiment 1/Civil_embedding/doc2vec_civil_all.npy")

## RandomForest

In [ ]:
# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    'n_estimators': [100, 150, 200, 250, 300, 350, 400, 450]
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

GridSearchCV(cv=7, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [100, 150, 200, 250, 300, 350, 400,
                                          450]},
             refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_n_estimators', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_n_estimators', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_n_estimators', 'mean_test_accuracy']])


Best Parameters: {'n_estimators': 400}
Best Score (Accuracy): 0.6033014344431508
Precision Scores:   param_n_estimators  mean_test_precision
0                100             0.590170
1                150             0.594523
2                200             0.599873
3                250             0.600444
4                300             0.603448
5                350             0.606124
6                400             0.609847
7                450             0.609986
Recall Scores:   param_n_estimators  mean_test_recall
0                100          0.454614
1                150          0.446605
2                200          0.446307
3                250          0.439347
4                300          0.436503
5                350          0.438673
6                400          0.440693
7                450          0.439272
Accuracy Scores:   param_n_estimators  mean_test_accuracy
0                100            0.593307
1                150            0.594830
2                

In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)


Best Precision Scores:
  param_n_estimators  mean_test_precision
6                400             0.609847

Best Recall Scores:
  param_n_estimators  mean_test_recall
6                400          0.440693

Best Accuracy Scores:
  param_n_estimators  mean_test_accuracy
6                400            0.603301


In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.61760084 0.61969618 0.62074384 0.61498167 0.617077   0.61079099
 0.62022001 0.61760084]
Fold 2 Recall: [0.39601886 0.38187533 0.37558931 0.37558931 0.37454164 0.37454164
 0.382923   0.37716082]
Fold 3 Recall: [0.37768465 0.37297014 0.37349398 0.35463594 0.34992142 0.36039811
 0.36144578 0.36196962]
Fold 4 Recall: [0.40806705 0.40440021 0.39549502 0.38344683 0.38135149 0.38658984
 0.38187533 0.37925616]
Fold 5 Recall: [0.37768465 0.35882661 0.35620744 0.35463594 0.34677842 0.35201676
 0.35463594 0.34992142]
Fold 6 Recall: [0.41509434 0.39937107 0.40303983 0.38888889 0.38469602 0.38731656
 0.3836478  0.38574423]
Fold 7 Recall: [0.59014675 0.58909853 0.59958071 0.60324948 0.60115304 0.5990566
 0.60010482 0.60324948]
Fold 1 Precision: [0.77565789 0.79556153 0.80722071 0.81189488 0.82782853 0.81997187
 0.82508711 0.82736842]
Fold 2 Precision: [0.54038599 0.53920118 0.54195011 0.54691076 0.55772231 0.55383424
 0.56274057 0.56118472]
Fold 3 Precision: [0.50774648 0.51594203 

In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


Best Accuracy: 0.7578764574547259
Corresponding Precision: 0.7756578947368421
Corresponding Recall: 0.6176008381351493


## ExtraTreesClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score

# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    'n_estimators': [100, 250, 400]
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(ExtraTreesClassifier(n_jobs=-1), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

GridSearchCV(cv=7, estimator=ExtraTreesClassifier(n_jobs=-1),
             param_grid={'n_estimators': [100, 250, 400]}, refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_n_estimators', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_n_estimators', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_n_estimators', 'mean_test_accuracy']])


Best Parameters: {'n_estimators': 400}
Best Score (Accuracy): 0.6384263890333784
Precision Scores:   param_n_estimators  mean_test_precision
0                100             0.629210
1                250             0.630586
2                400             0.638712
Recall Scores:   param_n_estimators  mean_test_recall
0                100          0.514644
1                250          0.515394
2                400          0.525048
Accuracy Scores:   param_n_estimators  mean_test_accuracy
0                100            0.629920
1                250            0.632011
2                400            0.638426


In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)


Best Precision Scores:
  param_n_estimators  mean_test_precision
2                400             0.638712

Best Recall Scores:
  param_n_estimators  mean_test_recall
2                400          0.525048

Best Accuracy Scores:
  param_n_estimators  mean_test_accuracy
2                400            0.638426


In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.7522263  0.78208486 0.78941854]
Fold 2 Recall: [0.4284966  0.43059193 0.44787847]
Fold 3 Recall: [0.4164484  0.40178104 0.41068622]
Fold 4 Recall: [0.41749607 0.39706653 0.4148769 ]
Fold 5 Recall: [0.40230487 0.39287585 0.39182818]
Fold 6 Recall: [0.4533543  0.44863732 0.46383648]
Fold 7 Recall: [0.73218029 0.75471698 0.75681342]
Fold 1 Precision: [0.87990196 0.89885611 0.9040192 ]
Fold 2 Precision: [0.55874317 0.55880354 0.5730563 ]
Fold 3 Precision: [0.52544613 0.51545699 0.53297077]
Fold 4 Precision: [0.50926518 0.48652118 0.49162011]
Fold 5 Precision: [0.50693069 0.49603175 0.49800266]
Fold 6 Precision: [0.58564658 0.6028169  0.61076605]
Fold 7 Precision: [0.83853541 0.85561497 0.86054827]
Fold 1 Accuracy: [0.83403622 0.8551228  0.8605805 ]
Fold 2 Accuracy: [0.56908956 0.56933763 0.58050112]
Fold 3 Accuracy: [0.5455222  0.5378318  0.55048375]
Fold 4 Accuracy: [0.53349876 0.51588089 0.51960298]
Fold 5 Accuracy: [0.53151365 0.52332506 0.5248139 ]
Fold 6 Accuracy: [0

In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


## XGBoost

In [ ]:
# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    'max_depth': [3, 5, 7, 9]
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(XGBClassifier(device='cuda'), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:35:04] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=7,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=Non...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'max_depth': [3, 5, 7, 9]}, refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_max_depth', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_max_depth', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_max_depth', 'mean_test_accuracy']])


Best Parameters: {'max_depth': 9}
Best Score (Accuracy): 0.5860399169815709
Precision Scores:   param_max_depth  mean_test_precision
0               3             0.525010
1               5             0.541087
2               7             0.557074
3               9             0.569058
Recall Scores:   param_max_depth  mean_test_recall
0               3          0.416887
1               5          0.486121
2               7          0.516283
3               9          0.525791
Accuracy Scores:   param_max_depth  mean_test_accuracy
0               3            0.544358
1               5            0.560662
2               7            0.575867
3               9            0.586040


In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_max_depth', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_max_depth', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_max_depth', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)


Best Precision Scores:
  param_max_depth  mean_test_precision
3               9             0.569058

Best Recall Scores:
  param_max_depth  mean_test_recall
3               9          0.525791

Best Accuracy Scores:
  param_max_depth  mean_test_accuracy
3               9             0.58604


In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.45887899 0.56678889 0.64798324 0.66841278]
Fold 2 Recall: [0.40230487 0.46411734 0.49607124 0.48664222]
Fold 3 Recall: [0.41173389 0.44368779 0.46883185 0.46254584]
Fold 4 Recall: [0.40073337 0.45783133 0.46254584 0.46097433]
Fold 5 Recall: [0.38920901 0.44054479 0.45364065 0.45625982]
Fold 6 Recall: [0.40461216 0.48165618 0.47222222 0.48689727]
Fold 7 Recall: [0.45073375 0.54821803 0.61268344 0.65880503]
Fold 1 Precision: [0.60041124 0.63910219 0.70605023 0.73928158]
Fold 2 Precision: [0.51440054 0.52087008 0.53502825 0.53025114]
Fold 3 Precision: [0.50546624 0.49852855 0.49694614 0.50399543]
Fold 4 Precision: [0.46703297 0.48963585 0.48067501 0.47670639]
Fold 5 Precision: [0.48689384 0.48361127 0.48218263 0.48632049]
Fold 6 Precision: [0.53094911 0.52725186 0.53630952 0.536684  ]
Fold 7 Precision: [0.56991385 0.62860577 0.66232295 0.71016949]
Fold 1 Accuracy: [0.59910692 0.6432647  0.70553213 0.73133218]
Fold 2 Accuracy: [0.53708757 0.54403374 0.55718184 0.55271645]

In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


Best Accuracy: 0.7313321756387993
Corresponding Precision: 0.6004112405757368
Corresponding Recall: 0.45887899423782086


## KNeighborsClassifier

In [ ]:
# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    'n_neighbors': [3, 5, 7, 9]
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

[I] [13:43:09.488062] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:43:09.494916] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:43:10.820587] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:43:10.968213] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:43:11.115056] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:43:11.242195] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:43:11.363809] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:43:11.491057] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:43:11.617319] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:43:11.740346] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:43:11.875964] Unused keyword parameter: n_jobs during cuML estimator ini

GridSearchCV(cv=7, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9]}, refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_n_neighbors', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_n_neighbors', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_n_neighbors', 'mean_test_accuracy']])


Best Parameters: {'n_neighbors': 3}
Best Score (Accuracy): 0.5923147538515533
Precision Scores:   param_n_neighbors  mean_test_precision
0                 3             0.572535
1                 5             0.551504
2                 7             0.543785
3                 9             0.537015
Recall Scores:   param_n_neighbors  mean_test_recall
0                 3          0.553859
1                 5          0.525417
2                 7          0.509248
3                 9          0.492857
Accuracy Scores:   param_n_neighbors  mean_test_accuracy
0                 3            0.592315
1                 5            0.572360
2                 7            0.564384
3                 9            0.557791


In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_n_neighbors', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_n_neighbors', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_n_neighbors', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)


Best Precision Scores:
  param_n_neighbors  mean_test_precision
0                 3             0.572535

Best Recall Scores:
  param_n_neighbors  mean_test_recall
0                 3          0.553859

Best Accuracy Scores:
  param_n_neighbors  mean_test_accuracy
0                 3            0.592315


In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.70560503 0.63436354 0.59664746 0.55578837]
Fold 2 Recall: [0.49973808 0.47354636 0.47040335 0.46516501]
Fold 3 Recall: [0.48088004 0.47773704 0.48664222 0.47040335]
Fold 4 Recall: [0.48926139 0.4772132  0.46568884 0.46149817]
Fold 5 Recall: [0.48821372 0.47197486 0.46883185 0.45364065]
Fold 6 Recall: [0.53144654 0.5115304  0.49318658 0.4827044 ]
Fold 7 Recall: [0.68186583 0.63155136 0.58333333 0.56079665]
Fold 1 Precision: [0.73366013 0.67615857 0.64715909 0.63192376]
Fold 2 Precision: [0.51960784 0.50166482 0.50791855 0.51418645]
Fold 3 Precision: [0.4901228  0.49377369 0.50516585 0.50336323]
Fold 4 Precision: [0.49365751 0.48952176 0.48315217 0.48353458]
Fold 5 Precision: [0.50323974 0.49369863 0.49611973 0.49485714]
Fold 6 Precision: [0.55653128 0.53922652 0.52511161 0.51770658]
Fold 7 Precision: [0.71092896 0.6664823  0.64186851 0.61353211]
Fold 1 Accuracy: [0.73927065 0.68295708 0.65492434 0.63631853]
Fold 2 Accuracy: [0.54428182 0.52790871 0.53336641 0.53857604]

In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


Best Accuracy: 0.7392706524435624
Corresponding Precision: 0.7336601307189542
Corresponding Recall: 0.7056050288108957


## QDA

In [ ]:
# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    # 'reg_param': [0.001, 0.01, 0.1, 1.0]
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(QuadraticDiscriminantAnalysis(), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

GridSearchCV(cv=7, estimator=QuadraticDiscriminantAnalysis(), param_grid={},
             refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,...,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,split5_test_accuracy,split6_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy
0,4.246386,1.76649,0.115123,0.064304,{},0.957775,0.554187,0.505484,0.492847,0.490892,...,0.967502,0.580997,0.532374,0.51861,0.516873,0.6134,0.916625,0.663769,0.17948,1


In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_reg_param', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_reg_param', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_reg_param', 'mean_test_accuracy']])


In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_reg_param', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_reg_param', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_reg_param', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)

In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.97433211]
Fold 2 Recall: [0.58931378]
Fold 3 Recall: [0.57936092]
Fold 4 Recall: [0.55945521]
Fold 5 Recall: [0.5364065]
Fold 6 Recall: [0.62997904]
Fold 7 Recall: [0.92872117]
Fold 1 Precision: [0.95777549]
Fold 2 Precision: [0.55418719]
Fold 3 Precision: [0.50548446]
Fold 4 Precision: [0.49284725]
Fold 5 Precision: [0.49089166]
Fold 6 Precision: [0.58519961]
Fold 7 Precision: [0.89858012]
Fold 1 Accuracy: [0.96750186]
Fold 2 Accuracy: [0.58099727]
Fold 3 Accuracy: [0.5323741]
Fold 4 Accuracy: [0.51861042]
Fold 5 Accuracy: [0.51687345]
Fold 6 Accuracy: [0.6133995]
Fold 7 Accuracy: [0.91662531]


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict


In [ ]:
best_rf = grid_search.best_estimator_



# Get cross-validated predictions
predicted = cross_val_predict(best_rf, X, y, cv=grid_search.cv)

# Calculate the confusion matrix for each fold
for fold_idx in range(grid_search.cv):
    start_idx = fold_idx * len(y) // grid_search.cv
    end_idx = (fold_idx + 1) * len(y) // grid_search.cv

    fold_labels = y[start_idx:end_idx]
    fold_predictions = predicted[start_idx:end_idx]

    fold_conf_matrix = confusion_matrix(fold_labels, fold_predictions)
    print(f"Confusion Matrix for Fold {fold_idx + 1}:\n", fold_conf_matrix)


Confusion Matrix for Fold 1:
 [[2053   83]
 [  48 1846]]
Confusion Matrix for Fold 2:
 [[1247  946]
 [ 746 1091]]
Confusion Matrix for Fold 3:
 [[1020 1077]
 [ 818 1116]]
Confusion Matrix for Fold 4:
 [[ 983 1035]
 [ 888 1124]]
Confusion Matrix for Fold 5:
 [[1055 1064]
 [ 887 1025]]
Confusion Matrix for Fold 6:
 [[1254  873]
 [ 681 1222]]
Confusion Matrix for Fold 7:
 [[1958  204]
 [ 136 1733]]


In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    # fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


Best Accuracy: 0.9675018605805011
Corresponding Precision: 0.9577754891864058
Corresponding Recall: 0.9743321110529073


# Criminal

Loading criminal dataset

In [ ]:
# loading dataset
dataset = pd.read_csv("/content/drive/MyDrive/LJP_2023/LJP_Dataset_Files/criminal_documents/case_files_criminal.csv")
print(len(dataset))
dataset.rename(columns = {'judgement':'text'}, inplace = True)
dataset = dataset[dataset['label'] != "Other" ]
dataset["label"] = dataset["label"].replace("Accepted", 1)
dataset["label"] = dataset["label"].replace("Rejected", 0)

#filter dataset
dataset.label.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset = dataset.dropna(axis=0, subset=['text'])
dataset = dataset.dropna(axis=0, subset=['label'])
dataset['label'] = dataset['label'].astype(int)
dataset.reset_index(drop=True, inplace=True)
dataset = dataset['label']
print(dataset) #should have only the label column

9069
0       0
1       0
2       0
3       1
4       0
       ..
7358    0
7359    0
7360    0
7361    1
7362    1
Name: label, Length: 7363, dtype: int64


In [ ]:
y=dataset

Loading criminal dataset embeddings

In [ ]:
X=np.load("/content/drive/MyDrive/LJP_2023/Embeddings/Doc2Vec/Experiment 1/Criminal_embedding/doc2vec_criminal_all.npy")

# Model

## RandomForest

In [ ]:
# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    'n_estimators': [100, 150, 200, 250, 300, 350, 400, 450]
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

GridSearchCV(cv=7, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [100, 150, 200, 250, 300, 350, 400,
                                          450]},
             refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_n_estimators', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_n_estimators', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_n_estimators', 'mean_test_accuracy']])


Best Parameters: {'n_estimators': 250}
Best Score (Accuracy): 0.6164773622906923
Precision Scores:   param_n_estimators  mean_test_precision
0                100             0.590587
1                150             0.605369
2                200             0.611610
3                250             0.616784
4                300             0.611679
5                350             0.611933
6                400             0.615055
7                450             0.608918
Recall Scores:   param_n_estimators  mean_test_recall
0                100          0.431876
1                150          0.434806
2                200          0.435979
3                250          0.436854
4                300          0.429823
5                350          0.432167
6                400          0.431292
7                450          0.427481
Accuracy Scores:   param_n_estimators  mean_test_accuracy
0                100            0.599907
1                150            0.609550
2                

In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)


Best Precision Scores:
  param_n_estimators  mean_test_precision
3                250             0.616784

Best Recall Scores:
  param_n_estimators  mean_test_recall
3                250          0.436854

Best Accuracy Scores:
  param_n_estimators  mean_test_accuracy
3                250            0.616477


In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.62628337 0.63655031 0.64887064 0.65913758 0.64065708 0.65092402
 0.64271047 0.64681725]
Fold 2 Recall: [0.35523614 0.36755647 0.36755647 0.34291581 0.34702259 0.34907598
 0.34702259 0.33880903]
Fold 3 Recall: [0.37577002 0.3613963  0.35112936 0.35112936 0.33880903 0.34291581
 0.34907598 0.34291581]
Fold 4 Recall: [0.35523614 0.34702259 0.36550308 0.35728953 0.36550308 0.35934292
 0.36755647 0.3613963 ]
Fold 5 Recall: [0.34631148 0.35245902 0.34836066 0.34631148 0.33196721 0.33401639
 0.32786885 0.32581967]
Fold 6 Recall: [0.38729508 0.39344262 0.39754098 0.41803279 0.40163934 0.40778689
 0.39754098 0.39139344]
Fold 7 Recall: [0.57700205 0.58521561 0.57289528 0.58316222 0.58316222 0.58110883
 0.58726899 0.58521561]
Fold 1 Precision: [0.81117021 0.81578947 0.83377309 0.8381201  0.83646113 0.84533333
 0.84823848 0.83554377]
Fold 2 Precision: [0.52108434 0.54573171 0.54407295 0.521875   0.528125   0.53291536
 0.53481013 0.52715655]
Fold 3 Precision: [0.5097493  0.52225519

In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


Best Accuracy: 0.7832699619771863
Corresponding Precision: 0.8111702127659575
Corresponding Recall: 0.6262833675564682


## ExtraTreesClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score

# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    'n_estimators': [100, 250, 400]
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(ExtraTreesClassifier(n_jobs=-1), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

GridSearchCV(cv=7, estimator=ExtraTreesClassifier(n_jobs=-1),
             param_grid={'n_estimators': [100, 250, 400]}, refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_n_estimators', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_n_estimators', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_n_estimators', 'mean_test_accuracy']])


Best Parameters: {'n_estimators': 400}
Best Score (Accuracy): 0.6352259636330935
Precision Scores:   param_n_estimators  mean_test_precision
0                100             0.625622
1                250             0.623689
2                400             0.623500
Recall Scores:   param_n_estimators  mean_test_recall
0                100          0.496966
1                250          0.496970
2                400          0.497261
Accuracy Scores:   param_n_estimators  mean_test_accuracy
0                100            0.634139
1                250            0.634819
2                400            0.635226


In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)


Best Precision Scores:
  param_n_estimators  mean_test_precision
2                400               0.6235

Best Recall Scores:
  param_n_estimators  mean_test_recall
2                400          0.497261

Best Accuracy Scores:
  param_n_estimators  mean_test_accuracy
2                400            0.635226


In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.74948665 0.78234086 0.7761807 ]
Fold 2 Recall: [0.42915811 0.40862423 0.40246407]
Fold 3 Recall: [0.34291581 0.35318275 0.35523614]
Fold 4 Recall: [0.40041068 0.38193018 0.37577002]
Fold 5 Recall: [0.36885246 0.3647541  0.35655738]
Fold 6 Recall: [0.47540984 0.46516393 0.48155738]
Fold 7 Recall: [0.71252567 0.72279261 0.73305955]
Fold 1 Precision: [0.85680751 0.87990762 0.87906977]
Fold 2 Precision: [0.56334232 0.55431755 0.54444444]
Fold 3 Precision: [0.48265896 0.49142857 0.49287749]
Fold 4 Precision: [0.52560647 0.51810585 0.52586207]
Fold 5 Precision: [0.46511628 0.48369565 0.46774194]
Fold 6 Precision: [0.65168539 0.59424084 0.63002681]
Fold 7 Precision: [0.83413462 0.8441247  0.82448037]
Fold 1 Accuracy: [0.82604563 0.84980989 0.84695817]
Fold 2 Accuracy: [0.58174905 0.57414449 0.56749049]
Fold 3 Accuracy: [0.5256654  0.53136882 0.53231939]
Fold 4 Accuracy: [0.55513308 0.54942966 0.55418251]
Fold 5 Accuracy: [0.51045627 0.52471483 0.51330798]
Fold 6 Accuracy: [0

In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


## XGBoost

In [ ]:
# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    'max_depth': [3, 5, 7, 9]
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(XGBClassifier(device='cuda'), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

GridSearchCV(cv=7,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=Non...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'max_depth': [3, 5, 7, 9]}, refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_max_depth', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_max_depth', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_max_depth', 'mean_test_accuracy']])


Best Parameters: {'max_depth': 9}
Best Score (Accuracy): 0.6004526869989059
Precision Scores:   param_max_depth  mean_test_precision
0               3             0.531663
1               5             0.561513
2               7             0.574693
3               9             0.576039
Recall Scores:   param_max_depth  mean_test_recall
0               3          0.453843
1               5          0.508093
2               7          0.508681
3               9          0.517487
Accuracy Scores:   param_max_depth  mean_test_accuracy
0               3            0.561195
1               5            0.589040
2               7            0.598958
3               9            0.600453


In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_max_depth', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_max_depth', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_max_depth', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)


Best Precision Scores:
  param_max_depth  mean_test_precision
3               9             0.576039

Best Recall Scores:
  param_max_depth  mean_test_recall
3               9          0.517487

Best Accuracy Scores:
  param_max_depth  mean_test_accuracy
3               9            0.600453


In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.57905544 0.64681725 0.724846   0.73716632]
Fold 2 Recall: [0.42915811 0.49486653 0.43737166 0.44353183]
Fold 3 Recall: [0.40041068 0.42915811 0.41067762 0.43942505]
Fold 4 Recall: [0.40862423 0.4476386  0.43942505 0.45174538]
Fold 5 Recall: [0.39754098 0.43647541 0.42827869 0.41188525]
Fold 6 Recall: [0.45081967 0.47540984 0.47745902 0.47540984]
Fold 7 Recall: [0.51129363 0.62628337 0.64271047 0.66324435]
Fold 1 Precision: [0.64827586 0.69690265 0.78444444 0.79249448]
Fold 2 Precision: [0.5        0.51059322 0.4965035  0.51551313]
Fold 3 Precision: [0.46428571 0.50483092 0.48780488 0.48636364]
Fold 4 Precision: [0.49257426 0.50230415 0.48526077 0.4954955 ]
Fold 5 Precision: [0.44803695 0.48409091 0.476082   0.44078947]
Fold 6 Precision: [0.52505967 0.55106888 0.56144578 0.56585366]
Fold 7 Precision: [0.64341085 0.68080357 0.73130841 0.7357631 ]
Fold 1 Accuracy: [0.65969582 0.70627376 0.78041825 0.78897338]
Fold 2 Accuracy: [0.53707224 0.54657795 0.53422053 0.54942966]

In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


Best Accuracy: 0.7889733840304183
Corresponding Precision: 0.6482758620689655
Corresponding Recall: 0.5790554414784395


## KNeighborsClassifier

In [ ]:
# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

[I] [13:53:25.799311] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:53:25.803007] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:53:25.838327] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:53:25.868261] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:53:25.897688] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:53:25.928737] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:53:25.960892] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:53:25.993074] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:53:26.022375] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:53:26.050186] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [13:53:26.078879] Unused keyword parameter: n_jobs during cuML estimator ini

GridSearchCV(cv=7, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9]}, refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_n_neighbors', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_n_neighbors', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_n_neighbors', 'mean_test_accuracy']])


Best Parameters: {'n_neighbors': 3}
Best Score (Accuracy): 0.5932551239320457
Precision Scores:   param_n_neighbors  mean_test_precision
0                 3             0.563841
1                 5             0.541472
2                 7             0.536750
3                 9             0.528074
Recall Scores:   param_n_neighbors  mean_test_recall
0                 3          0.540632
1                 5          0.500753
2                 7          0.478469
3                 9          0.461760
Accuracy Scores:   param_n_neighbors  mean_test_accuracy
0                 3            0.593255
1                 5            0.572063
2                 7            0.566493
3                 9            0.558885


In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_n_neighbors', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_n_neighbors', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_n_neighbors', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)


Best Precision Scores:
  param_n_neighbors  mean_test_precision
0                 3             0.563841

Best Recall Scores:
  param_n_neighbors  mean_test_recall
0                 3          0.540632

Best Accuracy Scores:
  param_n_neighbors  mean_test_accuracy
0                 3            0.593255


In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.67556468 0.60164271 0.57700205 0.56262834]
Fold 2 Recall: [0.54620123 0.48665298 0.4825462  0.45379877]
Fold 3 Recall: [0.42094456 0.40862423 0.40041068 0.39014374]
Fold 4 Recall: [0.47022587 0.46817248 0.42710472 0.43737166]
Fold 5 Recall: [0.47131148 0.43442623 0.42418033 0.40778689]
Fold 6 Recall: [0.51434426 0.49795082 0.47336066 0.45081967]
Fold 7 Recall: [0.68583162 0.60780287 0.56468172 0.52977413]
Fold 1 Precision: [0.71521739 0.65842697 0.66273585 0.65238095]
Fold 2 Precision: [0.54845361 0.53020134 0.5199115  0.49774775]
Fold 3 Precision: [0.46171171 0.46933962 0.46539379 0.45783133]
Fold 4 Precision: [0.495671   0.49890591 0.48372093 0.49191686]
Fold 5 Precision: [0.4742268  0.45788337 0.4579646  0.45852535]
Fold 6 Precision: [0.53177966 0.51483051 0.525      0.51643192]
Fold 7 Precision: [0.71982759 0.66071429 0.64252336 0.62168675]
Fold 1 Accuracy: [0.72528517 0.67110266 0.66825095 0.65874525]
Fold 2 Accuracy: [0.58174905 0.56273764 0.55418251 0.5351711 ]

In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    # fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


Best Accuracy: 0.730732635585157
Corresponding Precision: 0.7198275862068966
Corresponding Recall: 0.6858316221765913


## QDA

In [ ]:
# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    # 'reg_param': [0.001, 0.01, 0.1, 1.0]
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(QuadraticDiscriminantAnalysis(), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

GridSearchCV(cv=7, estimator=QuadraticDiscriminantAnalysis(), param_grid={},
             refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_reg_param', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_reg_param', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_reg_param', 'mean_test_accuracy']])


In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_reg_param', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_reg_param', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_reg_param', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)

In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [1.]
Fold 2 Recall: [0.40041068]
Fold 3 Recall: [0.36960986]
Fold 4 Recall: [0.33264887]
Fold 5 Recall: [0.32786885]
Fold 6 Recall: [0.47745902]
Fold 7 Recall: [0.92402464]
Fold 1 Precision: [1.]
Fold 2 Precision: [0.61320755]
Fold 3 Precision: [0.49861496]
Fold 4 Precision: [0.48942598]
Fold 5 Precision: [0.47761194]
Fold 6 Precision: [0.69552239]
Fold 7 Precision: [0.94936709]
Fold 1 Accuracy: [1.]
Fold 2 Accuracy: [0.60551331]
Fold 3 Accuracy: [0.53612167]
Fold 4 Accuracy: [0.53041825]
Fold 5 Accuracy: [0.52186312]
Fold 6 Accuracy: [0.66064639]
Fold 7 Accuracy: [0.94196004]


In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

# Assuming 'best_rf' is your best estimator
# Replace 'best_rf' with grid_search.best_estimator_ if needed
best_rf = grid_search.best_estimator_

# Assuming 'X' is your feature data and 'y' is your target data
# Replace 'X' and 'y' with your actual dataset


# Get cross-validated predictions
predicted = cross_val_predict(best_rf, X, y, cv=grid_search.cv)

# Calculate the confusion matrix for each fold
for fold_idx in range(grid_search.cv):
    start_idx = fold_idx * len(y) // grid_search.cv
    end_idx = (fold_idx + 1) * len(y) // grid_search.cv

    fold_labels = y[start_idx:end_idx]
    fold_predictions = predicted[start_idx:end_idx]

    fold_conf_matrix = confusion_matrix(fold_labels, fold_predictions)
    print(f"Confusion Matrix for Fold {fold_idx + 1}:\n", fold_conf_matrix)


Confusion Matrix for Fold 1:
 [[580   0]
 [  0 471]]
Confusion Matrix for Fold 2:
 [[434 125]
 [287 206]]
Confusion Matrix for Fold 3:
 [[403 187]
 [292 170]]
Confusion Matrix for Fold 4:
 [[350 154]
 [362 186]]
Confusion Matrix for Fold 5:
 [[376 167]
 [348 161]]
Confusion Matrix for Fold 6:
 [[493 117]
 [218 224]]
Confusion Matrix for Fold 7:
 [[542  24]
 [ 37 449]]


In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    # fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


Best Accuracy: 1.0
Corresponding Precision: 1.0
Corresponding Recall: 1.0


# Total

In [ ]:
# loading dataset
dataset = pd.read_csv("/content/drive/MyDrive/LJP_2023/LJP_Dataset_Files/total_documents/case_files_total.csv")
print(len(dataset))
dataset.rename(columns = {'judgement':'text'}, inplace = True)
dataset = dataset[dataset['label'] != "Other" ]
dataset["label"] = dataset["label"].replace("Accepted", 1)
dataset["label"] = dataset["label"].replace("Rejected", 0)

#filter dataset
dataset.label.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset = dataset.dropna(axis=0, subset=['text'])
dataset = dataset.dropna(axis=0, subset=['label'])
dataset['label'] = dataset['label'].astype(int)
dataset.reset_index(drop=True, inplace=True)
dataset = dataset['label']
print(dataset) #should have only the label column

53446
0        1
1        1
2        1
3        1
4        0
        ..
42337    0
42338    1
42339    1
42340    0
42341    1
Name: label, Length: 42342, dtype: int64


In [ ]:
y=dataset

In [ ]:
X=np.load("/content/drive/MyDrive/LJP_2023/Embeddings/Doc2Vec/Experiment 1/Total_embedding/doc2vec_total.npy")

# Model

## RandomForest

In [ ]:
# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    'n_estimators': [100, 150, 200, 250, 300, 350, 400, 450]
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

GridSearchCV(cv=7, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [100, 150, 200, 250, 300, 350, 400,
                                          450]},
             refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_n_estimators', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_n_estimators', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_n_estimators', 'mean_test_accuracy']])


Best Parameters: {'n_estimators': 350}
Best Score (Accuracy): 0.5964787076179455
Precision Scores:   param_n_estimators  mean_test_precision
0                100             0.587434
1                150             0.599257
2                200             0.603378
3                250             0.608980
4                300             0.609185
5                350             0.613616
6                400             0.610705
7                450             0.612407
Recall Scores:   param_n_estimators  mean_test_recall
0                100          0.375951
1                150          0.365211
2                200          0.364908
3                250          0.363899
4                300          0.359512
5                350          0.357949
6                400          0.354621
7                450          0.355175
Accuracy Scores:   param_n_estimators  mean_test_accuracy
0                100            0.585615
1                150            0.589960
2                

In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)


Best Precision Scores:
  param_n_estimators  mean_test_precision
5                350             0.613616

Best Recall Scores:
  param_n_estimators  mean_test_recall
5                350          0.357949

Best Accuracy Scores:
  param_n_estimators  mean_test_accuracy
5                350            0.596479


In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.5084686  0.50105857 0.50811574 0.51693719 0.51482004 0.51834862
 0.51623147 0.52258292]
Fold 2 Recall: [0.30991881 0.30038828 0.306036   0.3000353  0.29827038 0.29509354
 0.29156371 0.29015178]
Fold 3 Recall: [0.326509   0.30638899 0.29474056 0.28944582 0.28591599 0.28556301
 0.27779739 0.27744441]
Fold 4 Recall: [0.30638899 0.2919167  0.29332863 0.29262266 0.28097423 0.28273915
 0.27850335 0.27673844]
Fold 5 Recall: [0.32933286 0.32368514 0.32227321 0.3162725  0.30462407 0.30180021
 0.29968232 0.29544652]
Fold 6 Recall: [0.35757148 0.34415814 0.34062831 0.34027533 0.33745146 0.33498059
 0.32933286 0.33215672]
Fold 7 Recall: [0.49346982 0.48888104 0.48923403 0.49170491 0.49452877 0.48711613
 0.48923403 0.49170491]
Fold 1 Precision: [0.763646   0.786268   0.79514081 0.81073603 0.81736695 0.82343049
 0.82329769 0.82598996]
Fold 2 Precision: [0.52796152 0.54446577 0.54494029 0.55158988 0.54798962 0.54533594
 0.54701987 0.55130785]
Fold 3 Precision: [0.51820728 0.51820896

In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    # fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


Best Accuracy: 0.724747892213589
Corresponding Precision: 0.7636459989401165
Corresponding Recall: 0.5084685956245589


## ExtraTreesClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score

# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    'n_estimators': [400]
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(ExtraTreesClassifier(n_jobs=-1), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

GridSearchCV(cv=7, estimator=ExtraTreesClassifier(n_jobs=-1),
             param_grid={'n_estimators': [400]}, refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_n_estimators', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_n_estimators', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_n_estimators', 'mean_test_accuracy']])


Best Parameters: {'n_estimators': 400}
Best Score (Accuracy): 0.6318826030164091
Precision Scores:   param_n_estimators  mean_test_precision
0                400             0.629611
Recall Scores:   param_n_estimators  mean_test_recall
0                400          0.488036
Accuracy Scores:   param_n_estimators  mean_test_accuracy
0                400            0.631883


In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_n_estimators', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)


Best Precision Scores:
  param_n_estimators  mean_test_precision
0                400             0.629611

Best Recall Scores:
  param_n_estimators  mean_test_recall
0                400          0.488036

Best Accuracy Scores:
  param_n_estimators  mean_test_accuracy
0                400            0.631883


In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.75158786]
Fold 2 Recall: [0.40557713]
Fold 3 Recall: [0.3632192]
Fold 4 Recall: [0.36180727]
Fold 5 Recall: [0.37592658]
Fold 6 Recall: [0.44299329]
Fold 7 Recall: [0.71514296]
Fold 1 Precision: [0.87402544]
Fold 2 Precision: [0.57306733]
Fold 3 Precision: [0.51296112]
Fold 4 Precision: [0.48949379]
Fold 5 Precision: [0.50569801]
Fold 6 Precision: [0.60716014]
Fold 7 Precision: [0.84487073]
Fold 1 Accuracy: [0.83286494]
Fold 2 Accuracy: [0.58009588]
Fold 3 Accuracy: [0.54025459]
Fold 4 Accuracy: [0.5243842]
Fold 5 Accuracy: [0.53562572]
Fold 6 Accuracy: [0.60489337]
Fold 7 Accuracy: [0.80505952]


In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    # fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


## XGBoost

In [ ]:
# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    'max_depth': [3, 5, 7, 9],
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(XGBClassifier(device='cuda'), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

GridSearchCV(cv=7,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=Non...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'max_depth': [3, 5, 7, 9]}, refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_max_depth', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_max_depth', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_max_depth', 'mean_test_accuracy']])


Best Parameters: {'max_depth': 9}
Best Score (Accuracy): 0.5771362224561395
Precision Scores:   param_max_depth  mean_test_precision
0               3             0.516155
1               5             0.530229
2               7             0.537342
3               9             0.553361
Recall Scores:   param_max_depth  mean_test_recall
0               3          0.340206
1               5          0.435051
2               7          0.488700
3               9          0.503826
Accuracy Scores:   param_max_depth  mean_test_accuracy
0               3            0.541307
1               5            0.554652
2               7            0.563225
3               9            0.577136


In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_max_depth', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_max_depth', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_max_depth', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)


Best Precision Scores:
  param_max_depth  mean_test_precision
3               9             0.553361

Best Recall Scores:
  param_max_depth  mean_test_recall
3               9          0.503826

Best Accuracy Scores:
  param_max_depth  mean_test_accuracy
3               9            0.577136


In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.36873677 0.50352858 0.59738885 0.62455893]
Fold 2 Recall: [0.32968585 0.39710554 0.46099541 0.46276032]
Fold 3 Recall: [0.31944935 0.40840099 0.43805154 0.45534769]
Fold 4 Recall: [0.32897988 0.40945994 0.42887399 0.43628662]
Fold 5 Recall: [0.3243911  0.40522414 0.44828803 0.43981645]
Fold 6 Recall: [0.34557007 0.42499118 0.47299682 0.49099894]
Fold 7 Recall: [0.36463113 0.49664666 0.57430286 0.61701377]
Fold 1 Precision: [0.57926829 0.6148212  0.65569326 0.69140625]
Fold 2 Precision: [0.49339672 0.50111359 0.50541796 0.51736385]
Fold 3 Precision: [0.49589041 0.49255002 0.48344371 0.49387443]
Fold 4 Precision: [0.49390567 0.49551474 0.46498278 0.48205928]
Fold 5 Precision: [0.48064854 0.48093842 0.48696319 0.48089541]
Fold 6 Precision: [0.51310273 0.51519042 0.52859961 0.53665123]
Fold 7 Precision: [0.55687332 0.61147327 0.63629253 0.67127496]
Fold 1 Accuracy: [0.57877335 0.61960655 0.66440734 0.69350306]
Fold 2 Accuracy: [0.52752521 0.53248471 0.53628699 0.54620598]

In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    # fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


Best Accuracy: 0.6935030583567532
Corresponding Precision: 0.5792682926829268
Corresponding Recall: 0.3687367678193366


## KNeighborsClassifier

In [ ]:
# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

[I] [14:08:02.939133] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:08:02.947911] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:08:03.319758] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:08:03.579774] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:08:03.842518] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:08:05.260359] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:08:05.527480] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:08:05.796514] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:08:06.061725] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:08:06.325654] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:08:06.593179] Unused keyword parameter: n_jobs during cuML estimator ini

GridSearchCV(cv=7, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9]}, refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_n_neighbors', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_n_neighbors', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_n_neighbors', 'mean_test_accuracy']])


Best Parameters: {'n_neighbors': 3}
Best Score (Accuracy): 0.585685779381461
Precision Scores:   param_n_neighbors  mean_test_precision
0                 3             0.559979
1                 5             0.543187
2                 7             0.534177
3                 9             0.527317
Recall Scores:   param_n_neighbors  mean_test_recall
0                 3          0.537860
1                 5          0.508062
2                 7          0.491070
3                 9          0.471860
Accuracy Scores:   param_n_neighbors  mean_test_accuracy
0                 3            0.585686
1                 5            0.569366
2                 7            0.560887
3                 9            0.554274


In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_n_neighbors', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_n_neighbors', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_n_neighbors', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)


Best Precision Scores:
  param_n_neighbors  mean_test_precision
0                 3             0.559979

Best Recall Scores:
  param_n_neighbors  mean_test_recall
0                 3           0.53786

Best Accuracy Scores:
  param_n_neighbors  mean_test_accuracy
0                 3            0.585686


In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.69230769 0.62032463 0.5843331  0.55292872]
Fold 2 Recall: [0.49452877 0.47687963 0.46487822 0.4511119 ]
Fold 3 Recall: [0.48252736 0.45005295 0.44722909 0.44405224]
Fold 4 Recall: [0.45534769 0.44193435 0.4306389  0.41404871]
Fold 5 Recall: [0.47370279 0.46911401 0.45287681 0.43769855]
Fold 6 Recall: [0.51394282 0.50123544 0.48429227 0.46417226]
Fold 7 Recall: [0.65266502 0.59689375 0.57324391 0.53900459]
Fold 1 Precision: [0.71605839 0.66767945 0.63423975 0.61595912]
Fold 2 Precision: [0.52550638 0.5188172  0.52282652 0.5112    ]
Fold 3 Precision: [0.49368003 0.48076923 0.48193229 0.48608964]
Fold 4 Precision: [0.47813195 0.47586469 0.46671767 0.4607227 ]
Fold 5 Precision: [0.48853295 0.49131238 0.48323917 0.4815534 ]
Fold 6 Precision: [0.53866075 0.53143713 0.52446483 0.52684295]
Fold 7 Precision: [0.67927994 0.63643207 0.62581888 0.60885167]
Fold 1 Accuracy: [0.72722764 0.67746735 0.64737973 0.62902959]
Fold 2 Accuracy: [0.55414118 0.54785915 0.55066953 0.54091585]

In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    # fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


Best Accuracy: 0.7272276409323856
Corresponding Precision: 0.7160583941605839
Corresponding Recall: 0.6923076923076923


## QDA

In [ ]:
# Assuming you have a RandomForestClassifier instance 'rf_classifier' and your data 'X', 'y'
param_grid = {
    # 'reg_param': [0.001, 0.01, 0.1, 1.0]
}

# Create a dictionary of scoring metrics
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'accuracy': make_scorer(accuracy_score)
}

grid_search = GridSearchCV(QuadraticDiscriminantAnalysis(), param_grid, cv=7, scoring=scoring, refit='accuracy')
grid_search.fit(X, y)

GridSearchCV(cv=7, estimator=QuadraticDiscriminantAnalysis(), param_grid={},
             refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

### scores

In [ ]:
best_rf = grid_search.best_estimator_

# Getting the results
results_df = pd.DataFrame(grid_search.cv_results_)

# Printing precision, recall, and accuracy scores
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Accuracy):", grid_search.best_score_)

# Accessing precision, recall, and accuracy scores from results_df
print("Precision Scores:", results_df[['param_reg_param', 'mean_test_precision']])
print("Recall Scores:", results_df[['param_reg_param', 'mean_test_recall']])
print("Accuracy Scores:", results_df[['param_reg_param', 'mean_test_accuracy']])


Best Parameters: {}
Best Score (Accuracy): 0.647967450283506


KeyError: ignored

In [ ]:
# Printing precision, recall, and accuracy scores for the best parameters
best_precision = results_df[results_df['rank_test_accuracy'] == 1][['param_reg_param', 'mean_test_precision']]
best_recall = results_df[results_df['rank_test_accuracy'] == 1][['param_reg_param', 'mean_test_recall']]
best_accuracy = results_df[results_df['rank_test_accuracy'] == 1][['param_reg_param', 'mean_test_accuracy']]

print("\nBest Precision Scores:")
print(best_precision)

print("\nBest Recall Scores:")
print(best_recall)

print("\nBest Accuracy Scores:")
print(best_accuracy)

In [ ]:
# To access recall scores for each fold (for demonstration purposes)
results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_recall_key = f"split{fold_idx}_test_recall"
    fold_recall = results[fold_recall_key]
    print(f"Fold {fold_idx + 1} Recall:", fold_recall)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_precision_key = f"split{fold_idx}_test_precision"
    fold_precision = results[fold_precision_key]
    print(f"Fold {fold_idx + 1} Precision:", fold_precision)

results = grid_search.cv_results_
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]
    print(f"Fold {fold_idx + 1} Accuracy:", fold_accuracy)

Fold 1 Recall: [0.950247]
Fold 2 Recall: [0.56088952]
Fold 3 Recall: [0.57359689]
Fold 4 Recall: [0.53335687]
Fold 5 Recall: [0.55594776]
Fold 6 Recall: [0.6244264]
Fold 7 Recall: [0.90187081]
Fold 1 Precision: [0.92005466]
Fold 2 Precision: [0.53375882]
Fold 3 Precision: [0.4789272]
Fold 4 Precision: [0.47575567]
Fold 5 Precision: [0.48357384]
Fold 6 Precision: [0.58770764]
Fold 7 Precision: [0.86084906]
Fold 1 Accuracy: [0.93800628]
Fold 2 Accuracy: [0.56488676]
Fold 3 Accuracy: [0.50801785]
Fold 4 Accuracy: [0.50619937]
Fold 5 Accuracy: [0.51396925]
Fold 6 Accuracy: [0.61894528]
Fold 7 Accuracy: [0.88574735]


In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

# Assuming 'best_rf' is your best estimator
# Replace 'best_rf' with grid_search.best_estimator_ if needed
best_rf = grid_search.best_estimator_

# Get cross-validated predictions
predicted = cross_val_predict(best_rf, X, y, cv=grid_search.cv)

# Calculate the confusion matrix for each fold
for fold_idx in range(grid_search.cv):
    start_idx = fold_idx * len(y) // grid_search.cv
    end_idx = (fold_idx + 1) * len(y) // grid_search.cv

    fold_labels = y[start_idx:end_idx]
    fold_predictions = predicted[start_idx:end_idx]

    fold_conf_matrix = confusion_matrix(fold_labels, fold_predictions)
    print(f"Confusion Matrix for Fold {fold_idx + 1}:\n", fold_conf_matrix)


Confusion Matrix for Fold 1:
 [[3017  236]
 [ 139 2656]]
Confusion Matrix for Fold 2:
 [[1826 1441]
 [1202 1580]]
Confusion Matrix for Fold 3:
 [[1452 1753]
 [1218 1626]]
Confusion Matrix for Fold 4:
 [[1471 1578]
 [1390 1610]]
Confusion Matrix for Fold 5:
 [[1535 1687]
 [1266 1561]]
Confusion Matrix for Fold 6:
 [[1971 1281]
 [1025 1772]]
Confusion Matrix for Fold 7:
 [[2847  415]
 [ 275 2512]]


In [ ]:
# Initialize variables to store the best accuracy and its corresponding fold index
best_accuracy = 0.0
best_fold_index = -1

# Iterate through each fold to find the one with the highest accuracy
for fold_idx in range(grid_search.cv):
    fold_accuracy_key = f"split{fold_idx}_test_accuracy"
    fold_accuracy = results[fold_accuracy_key]

    max_fold_accuracy = max(fold_accuracy)
    if max_fold_accuracy > best_accuracy:
        best_accuracy = max_fold_accuracy
        best_fold_index = fold_idx

# If a fold with the highest accuracy is found, retrieve its precision, recall, and parameters
if best_fold_index != -1:
    fold_precision_key = f"split{best_fold_index}_test_precision"
    fold_recall_key = f"split{best_fold_index}_test_recall"
    # fold_params = results['params'][best_fold_index]

    best_fold_precision = results[fold_precision_key][best_accuracy.argmax()]
    best_fold_recall = results[fold_recall_key][best_accuracy.argmax()]

    # Printing the best accuracy value, its corresponding precision, recall, and parameters
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Corresponding Precision: {best_fold_precision}")
    print(f"Corresponding Recall: {best_fold_recall}")
    # print(f"Parameters for Best Accuracy: {fold_params}")
else:
    print("No fold found with the highest accuracy.")


Best Accuracy: 0.9380062820300876
Corresponding Precision: 0.9200546634779638
Corresponding Recall: 0.9502470007057163
